In [27]:
!pip cache purge

Files removed: 2


In [1]:
!pip install datasets transformers accelerate evaluate

In [2]:
import datasets
import torch

import evaluate

from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from transformers import AutoTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [3]:
torch.manual_seed(123)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
  tokenizer = AutoTokenizer.from_pretrained("rasyosef/bert-mini-domain-adapted-imdb")
  model = AutoModelForSequenceClassification.from_pretrained("rasyosef/bert-mini-domain-adapted-imdb", num_labels=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/44.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rasyosef/bert-mini-domain-adapted-imdb and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model = model.to(device)

In [7]:
dataset = datasets.load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256, return_tensors='pt').to(device)

In [9]:
train = dataset['train'].map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [10]:
val = dataset['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    per_device_train_batch_size=64
    )

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=391, training_loss=0.5050866792878836, metrics={'train_runtime': 68.988, 'train_samples_per_second': 362.382, 'train_steps_per_second': 5.668, 'total_flos': 123892684800000.0, 'train_loss': 0.5050866792878836, 'epoch': 1.0})

In [14]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, device='cuda'):
        self.encodings = encodings
        self.labels = labels
        self.device = device

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(self.device)
        return item

    def __len__(self):
        return len(self.labels)

In [15]:
labels = val['label']
clear_val = val.map(remove_columns=['text', 'label'])
clear_val.set_format('pt')

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [16]:
clear_val

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [17]:
val_loader = DataLoader(clear_val, batch_size=64)

In [18]:
predict = []
for batch in val_loader:
  output = model(input_ids=batch['input_ids'].to(device),
                 token_type_ids=batch['token_type_ids'].to(device),
                 attention_mask=batch['attention_mask'].to(device))
  predict.extend(torch.argmax(output.logits, dim=1))

In [19]:
acc = evaluate.load('accuracy')

In [20]:
print(acc.compute(predictions=predict, references=labels))

{'accuracy': 0.8506}
